<div align = "center">

# **Siniestros**

</div>

## Librerias

In [2]:
import pandas as pd

## Data

In [3]:
siniestros = pd.read_csv("../data/in/SiniesCumpli.csv", sep=';')

## Conversión de Tipos de Datos

In [4]:
# Convertir columnas numéricas que están como strings
siniestros['VALOR_ASEGURADO'] = pd.to_numeric(siniestros['VALOR_ASEGURADO'], errors='coerce')
siniestros['RESERVA_ACTUAL_EQUI'] = pd.to_numeric(siniestros['RESERVA_ACTUAL_EQUI'], errors='coerce')
siniestros['PAGOS'] = pd.to_numeric(siniestros['PAGOS'], errors='coerce')

print(f"Dimensiones originales: {siniestros.shape}")
print(f"\nTipos de datos:")
print(siniestros[['VALOR_ASEGURADO', 'RESERVA_ACTUAL_EQUI', 'PAGOS']].dtypes)

Dimensiones originales: (202984, 15)

Tipos de datos:
VALOR_ASEGURADO        float64
RESERVA_ACTUAL_EQUI    float64
PAGOS                  float64
dtype: object


## Eliminación de Columnas Innecesarias

In [5]:
# Definir columnas a eliminar
columnas_eliminar = ['RAMO', 'Conta', 'anio', 'Mes', 'SINIESTRO', 'ULT_FECHA_PAGO']

# Eliminar columnas
siniestros_limpio = siniestros.drop(columns=columnas_eliminar)

print(f"Columnas eliminadas: {columnas_eliminar}")
print(f"\nColumnas restantes: {list(siniestros_limpio.columns)}")
print(f"Dimensiones después de eliminar columnas: {siniestros_limpio.shape}")

Columnas eliminadas: ['RAMO', 'Conta', 'anio', 'Mes', 'SINIESTRO', 'ULT_FECHA_PAGO']

Columnas restantes: ['cod_suc', 'POLIZA', 'DEPARTAMENTO_SINIESTRO', 'VALOR_ASEGURADO', 'FECHA_DE_SINIESTRO', 'FECHA_AVISO', 'AMPARO', 'PAGOS', 'RESERVA_ACTUAL_EQUI']
Dimensiones después de eliminar columnas: (202984, 9)


## Consolidación por POLIZA-AMPARO

In [6]:
# Consolidar por POLIZA-AMPARO
# MAX para valores numéricos (VALOR_ASEGURADO, RESERVA_ACTUAL_EQUI)
# SUM para PAGOS (suma total de todos los pagos históricos)
# FIRST para valores constantes (DEPARTAMENTO, FECHAS, cod_suc)
siniestros_consolidado = siniestros_limpio.groupby(['POLIZA', 'AMPARO']).agg({
    'VALOR_ASEGURADO': 'max',
    'RESERVA_ACTUAL_EQUI': 'max',
    'PAGOS': 'sum',
    'DEPARTAMENTO_SINIESTRO': 'first',
    'FECHA_DE_SINIESTRO': 'first',
    'FECHA_AVISO': 'first',
    'cod_suc': 'first'
}).reset_index()

print(f"Dimensiones consolidadas: {siniestros_consolidado.shape}")
print(f"\nCombinaciones únicas POLIZA-AMPARO: {siniestros_consolidado.shape[0]}")

Dimensiones consolidadas: (5746, 9)

Combinaciones únicas POLIZA-AMPARO: 5746


## Normalización de AMPARO en Siniestros

In [7]:
# Normalización de AMPARO truncados/abreviados
print("=" * 80)
print("NORMALIZACIÓN DE AMPARO EN SINIESTROS")
print("=" * 80)

# Mapeo de amparos truncados a nombres completos
mapeo_amparos = {
    'CALIDAD Y CORRECTO FUNCIONAMIE': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
    'CALIDAD Y BUEN FUNC.': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
}

# Mostrar amparos antes de normalizar
print("\nAmparos únicos ANTES de normalizar:")
print(siniestros_consolidado['AMPARO'].value_counts())

# Contar registros que serán normalizados
registros_a_normalizar = siniestros_consolidado['AMPARO'].isin(mapeo_amparos.keys()).sum()
print(f"\nRegistros a normalizar: {registros_a_normalizar:,}")

# Aplicar normalización
siniestros_consolidado['AMPARO'] = siniestros_consolidado['AMPARO'].replace(mapeo_amparos)

# Mostrar amparos después de normalizar
print("\nAmparos únicos DESPUÉS de normalizar:")
print(siniestros_consolidado['AMPARO'].value_counts())

NORMALIZACIÓN DE AMPARO EN SINIESTROS

Amparos únicos ANTES de normalizar:
AMPARO
CUMPLIMIENTO                      3745
BUEN MANEJO DEL ANTICIPO           731
PRESTACIONES SOCIALES              540
ESTABILIDAD DE LA OBRA             259
DISPOSICIONES LEGALES              238
CALIDAD DEL SERVICIO                60
SERIEDAD DE LA OFERTA               48
CALIDAD DE LOS ELEMENTOS            36
CALIDAD Y CORRECTO FUNCIONAMIE      31
CALIDAD                             28
PAGO ANTICIPADO                     26
CORRECTO FUNCIONAMIENTO              2
CALIDAD Y BUEN FUNC.                 1
PROVISION DE REPUESTOS               1
Name: count, dtype: int64

Registros a normalizar: 32

Amparos únicos DESPUÉS de normalizar:
AMPARO
CUMPLIMIENTO                         3745
BUEN MANEJO DEL ANTICIPO              731
PRESTACIONES SOCIALES                 540
ESTABILIDAD DE LA OBRA                259
DISPOSICIONES LEGALES                 238
CALIDAD DEL SERVICIO                   60
SERIEDAD DE LA OFERT

In [8]:
# =============================================================================
# CONVERSIÓN DE FECHA DE SINIESTRO
# =============================================================================
print("=" * 80)
print("CONVERSIÓN DE FECHA DE SINIESTRO")
print("=" * 80)

# Convertir fecha de siniestro a datetime si no está convertida
if 'FECHA_SINIESTRO_DT' not in siniestros_consolidado.columns:
    siniestros_consolidado['FECHA_SINIESTRO_DT'] = pd.to_datetime(
        siniestros_consolidado['FECHA_DE_SINIESTRO'],
        format='%d/%m/%Y',
        errors='coerce'
    )

print(f"\nTotal siniestros: {len(siniestros_consolidado):,}")
print(f"Con fecha válida: {siniestros_consolidado['FECHA_SINIESTRO_DT'].notna().sum():,}")

# Mostrar rango de fechas
print(f"\nRango de fechas de siniestros:")
print(f"  Mínima: {siniestros_consolidado['FECHA_SINIESTRO_DT'].min()}")
print(f"  Máxima: {siniestros_consolidado['FECHA_SINIESTRO_DT'].max()}")

# Nota: El filtro temporal se aplicará DESPUÉS del cruce con expuestos

CONVERSIÓN DE FECHA DE SINIESTRO

Total siniestros: 5,746
Con fecha válida: 5,746

Rango de fechas de siniestros:
  Mínima: 1995-08-01 00:00:00
  Máxima: 2025-06-24 00:00:00


## Verificación y Validación de Resultados

In [9]:
# Verificar que no hay duplicados
duplicados = siniestros_consolidado.duplicated(subset=['POLIZA', 'AMPARO']).sum()
print(f"Número de duplicados en (POLIZA, AMPARO): {duplicados}")

# Información del dataframe consolidado
print(f"\nInformación del dataframe consolidado:")
print(siniestros_consolidado.info())

# Estadísticas descriptivas de columnas numéricas
print(f"\nEstadísticas de VALOR_ASEGURADO:")
print(siniestros_consolidado['VALOR_ASEGURADO'].describe())

print(f"\nEstadísticas de RESERVA_ACTUAL_EQUI:")
print(siniestros_consolidado['RESERVA_ACTUAL_EQUI'].describe())

print(f"\nEstadísticas de PAGOS:")
print(siniestros_consolidado['PAGOS'].describe())

Número de duplicados en (POLIZA, AMPARO): 0

Información del dataframe consolidado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5746 entries, 0 to 5745
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   POLIZA                  5746 non-null   int64         
 1   AMPARO                  5746 non-null   object        
 2   VALOR_ASEGURADO         5549 non-null   float64       
 3   RESERVA_ACTUAL_EQUI     5643 non-null   float64       
 4   PAGOS                   5746 non-null   float64       
 5   DEPARTAMENTO_SINIESTRO  5746 non-null   object        
 6   FECHA_DE_SINIESTRO      5746 non-null   object        
 7   FECHA_AVISO             5746 non-null   object        
 8   cod_suc                 5746 non-null   int64         
 9   FECHA_SINIESTRO_DT      5746 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 449.0+ KB
None



In [10]:
# Mostrar primeras filas del dataframe consolidado
siniestros_consolidado.head(10)

,POLIZA,AMPARO,VALOR_ASEGURADO,RESERVA_ACTUAL_EQUI,PAGOS,DEPARTAMENTO_SINIESTRO,FECHA_DE_SINIESTRO,FECHA_AVISO,cod_suc,FECHA_SINIESTRO_DT
0,1450,CALIDAD,5.460002e+08,546000205.0,6.093379e+07,13-BOLIVAR,20/06/1999,14/09/1999,11,1999-06-20
1,6232,BUEN MANEJO DEL ANTICIPO,2.849548e+08,16893479.0,0.000000e+00,8-ATLANTICO,7/05/1998,7/09/1999,61,1998-05-07
2,6296,CUMPLIMIENTO,1.847356e+08,54000000.0,0.000000e+00,68-SANTANDER,20/10/1998,20/10/1998,51,1998-10-20
3,10309,PRESTACIONES SOCIALES,1.200000e+09,166481594.0,0.000000e+00,76-VALLE,4/05/1999,10/09/2001,21,1999-05-04
4,14213,CUMPLIMIENTO,1.997010e+08,0.0,1.049745e+08,11-BOGOTA D.C.,5/08/2002,15/11/2002,11,2002-08-05
5,31225,PRESTACIONES SOCIALES,7.165236e+07,58270562.0,0.000000e+00,23-CORDOBA,25/04/1999,5/03/2003,11,1999-04-25
6,62157,DISPOSICIONES LEGALES,1.969916e+07,19699000.0,0.000000e+00,8-ATLANTICO,12/08/1998,21/08/1998,61,1998-08-12
7,62557,PRESTACIONES SOCIALES,NaN,42120000.0,0.000000e+00,81-ARAUCA,10/04/2008,28/07/2009,51,2008-04-10
8,62830,DISPOSICIONES LEGALES,1.073063e+08,107306000.0,0.000000e+00,8-ATLANTICO,2/10/1998,14/07/1999,61,1998-10-02
9,62990,BUEN MANEJO DEL ANTICIPO,2.259262e+08,85029028.0,1.352460e+07,68-SANTANDER,5/10/2007,19/11/2007,51,2007-10-05


## Comparación: Antes vs Después

In [11]:
# Comparación de dimensiones
print("=" * 60)
print("COMPARACIÓN: ANTES vs DESPUÉS DE LA LIMPIEZA")
print("=" * 60)

print(f"\nDataframe original:")
print(f"  - Filas: {siniestros.shape[0]:,}")
print(f"  - Columnas: {siniestros.shape[1]}")

print(f"\nDataframe consolidado:")
print(f"  - Filas: {siniestros_consolidado.shape[0]:,}")
print(f"  - Columnas: {siniestros_consolidado.shape[1]}")

print(f"\nReducción de datos:")
print(f"  - Filas eliminadas: {siniestros.shape[0] - siniestros_consolidado.shape[0]:,}")
print(f"  - Porcentaje de reducción: {((siniestros.shape[0] - siniestros_consolidado.shape[0]) / siniestros.shape[0] * 100):.2f}%")
print(f"  - Columnas eliminadas: {siniestros.shape[1] - siniestros_consolidado.shape[1]}")

print(f"\nColumnas finales: {list(siniestros_consolidado.columns)}")

COMPARACIÓN: ANTES vs DESPUÉS DE LA LIMPIEZA

Dataframe original:
  - Filas: 202,984
  - Columnas: 15

Dataframe consolidado:
  - Filas: 5,746
  - Columnas: 10

Reducción de datos:
  - Filas eliminadas: 197,238
  - Porcentaje de reducción: 97.17%
  - Columnas eliminadas: 5

Columnas finales: ['POLIZA', 'AMPARO', 'VALOR_ASEGURADO', 'RESERVA_ACTUAL_EQUI', 'PAGOS', 'DEPARTAMENTO_SINIESTRO', 'FECHA_DE_SINIESTRO', 'FECHA_AVISO', 'cod_suc', 'FECHA_SINIESTRO_DT']


## Carga y Limpieza de Expuestos

In [12]:
# Cargar dataframe de expuestos
expuestos = pd.read_csv('../data/in/ProdCumpli.csv')

print(f"Dimensiones de expuestos: {expuestos.shape}")
print(f"\nColumnas:")
print(expuestos.dtypes)
print(f"\nPrimeras filas:")
expuestos.head()

# Crear copia para limpieza (todos los registros son Jun 2025)
expuestos_limpio = expuestos.copy()
print(f"\nexpuestos_limpio creado: {expuestos_limpio.shape[0]:,} registros")

Dimensiones de expuestos: (1242477, 10)

Columnas:
conta                     int64
anio                      int64
Mes                       int64
Poliza                    int64
CodSucursal               int64
Ramo                     object
VigenciaInicioPoliza     object
VigenciaFinPoliza        object
ValorPrimaEquiv         float64
Amparo                   object
dtype: object

Primeras filas:

expuestos_limpio creado: 1,242,477 registros


## Conversión de Fechas de Vigencia

In [13]:
# Convertir fechas de vigencia a datetime
expuestos_limpio['VIGENCIA_INICIO_DT'] = pd.to_datetime(expuestos_limpio['VigenciaInicioPoliza'], format='%d/%m/%Y')
expuestos_limpio['VIGENCIA_FIN_DT'] = pd.to_datetime(expuestos_limpio['VigenciaFinPoliza'], format='%d/%m/%Y')

# Calcular duración en días
expuestos_limpio['DURACION_DIAS'] = (expuestos_limpio['VIGENCIA_FIN_DT'] - expuestos_limpio['VIGENCIA_INICIO_DT']).dt.days + 1

print(f"Fechas convertidas a datetime")
print(f"\nEstadísticas de duración de vigencias:")
print(expuestos_limpio['DURACION_DIAS'].describe())

print(f"\nVigencias con duración inválida (FIN < INICIO):")
vigencias_invalidas = expuestos_limpio[expuestos_limpio['VIGENCIA_FIN_DT'] < expuestos_limpio['VIGENCIA_INICIO_DT']]
print(f"  Total: {len(vigencias_invalidas):,}")

Fechas convertidas a datetime

Estadísticas de duración de vigencias:
count    1.242477e+06
mean     8.945251e+02
std      6.611730e+02
min      1.000000e+00
25%      1.660000e+02
50%      1.126000e+03
75%      1.295000e+03
max      2.852800e+04
Name: DURACION_DIAS, dtype: float64

Vigencias con duración inválida (FIN < INICIO):
  Total: 0


In [14]:
# =============================================================================
# NORMALIZACIÓN DE AMPARO EN EXPUESTOS
# =============================================================================
# NOTA: El filtro de fechas 2010-2019 se aplica DESPUÉS del particionamiento
# para filtrar períodos anuales, no pólizas completas.
print("=" * 80)
print("NORMALIZACIÓN DE AMPARO EN EXPUESTOS")
print("=" * 80)

mapeo_amparos = {
    'CALIDAD Y CORRECTO FUNCIONAMIE': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
    'CALIDAD Y BUEN FUNC.': 'CALIDAD Y CORRECTO FUNCIONAMIENTO',
}

# Contar registros afectados ANTES de normalizar
registros_a_normalizar = expuestos_limpio['Amparo'].isin(mapeo_amparos.keys()).sum()
print(f"\nRegistros de expuestos con AMPARO a normalizar: {registros_a_normalizar:,}")

# Mostrar valores únicos antes
print(f"\nValores de AMPARO únicos ANTES (muestra de los que cambian):")
for amparo_viejo in mapeo_amparos.keys():
    count = (expuestos_limpio['Amparo'] == amparo_viejo).sum()
    if count > 0:
        print(f"  '{amparo_viejo}': {count:,} registros")

# Aplicar normalización
expuestos_limpio['Amparo'] = expuestos_limpio['Amparo'].replace(mapeo_amparos)

print(f"\n✅ Normalización completada")
print(f"Total registros en expuestos_limpio: {len(expuestos_limpio):,}")

NORMALIZACIÓN DE AMPARO EN EXPUESTOS

Registros de expuestos con AMPARO a normalizar: 0

Valores de AMPARO únicos ANTES (muestra de los que cambian):

✅ Normalización completada
Total registros en expuestos_limpio: 1,242,477


## Consolidación de Vigencias Múltiples por POLIZA-AMPARO

In [15]:
# =============================================================================
# CONSOLIDACIÓN DE VIGENCIAS MÚLTIPLES POR POLIZA-AMPARO
# =============================================================================
# Algunas pólizas tienen múltiples vigencias para el mismo amparo (prórrogas,
# endosos, etc.). Consolidamos tomando el rango temporal completo.
print("=" * 80)
print("CONSOLIDACIÓN DE VIGENCIAS MÚLTIPLES")
print("=" * 80)

print(f"\nAntes de consolidar: {len(expuestos_limpio):,} registros")

# Verificar duplicados
duplicados = expuestos_limpio.groupby(['Poliza', 'Amparo']).size()
polizas_con_multiples = (duplicados > 1).sum()
print(f"Combinaciones POLIZA-AMPARO con múltiples vigencias: {polizas_con_multiples:,}")

# Consolidar por POLIZA-AMPARO
expuestos_consolidado = expuestos_limpio.groupby(['Poliza', 'Amparo']).agg({
    'VIGENCIA_INICIO_DT': 'min',  # Fecha inicio más temprana
    'VIGENCIA_FIN_DT': 'max',      # Fecha fin más tardía
    'ValorPrimaEquiv': 'sum',      # Suma de exposiciones
    'CodSucursal': 'first',        # Primer valor
    'VigenciaInicioPoliza': 'first',  # Para referencia
    'VigenciaFinPoliza': 'last',      # Para referencia
}).reset_index()

# Renombrar columnas para consistencia
expuestos_consolidado = expuestos_consolidado.rename(columns={
    'Poliza': 'Poliza',
    'Amparo': 'Amparo'
})

# Recalcular duración
expuestos_consolidado['DURACION_DIAS'] = (
    expuestos_consolidado['VIGENCIA_FIN_DT'] - expuestos_consolidado['VIGENCIA_INICIO_DT']
).dt.days + 1

print(f"Después de consolidar: {len(expuestos_consolidado):,} registros")
print(f"Reducción: {len(expuestos_limpio) - len(expuestos_consolidado):,} registros ({100*(len(expuestos_limpio) - len(expuestos_consolidado))/len(expuestos_limpio):.2f}%)")

# Reemplazar expuestos_limpio con la versión consolidada
expuestos_limpio = expuestos_consolidado.copy()

print(f"\nEstadísticas de duración (post-consolidación):")
print(expuestos_limpio['DURACION_DIAS'].describe())

CONSOLIDACIÓN DE VIGENCIAS MÚLTIPLES

Antes de consolidar: 1,242,477 registros


Combinaciones POLIZA-AMPARO con múltiples vigencias: 251,328
Después de consolidar: 607,407 registros
Reducción: 635,070 registros (51.11%)

Estadísticas de duración (post-consolidación):
count    607407.000000
mean       1243.320520
std         643.233118
min           2.000000
25%         915.000000
50%        1219.000000
75%        1609.000000
max       28528.000000
Name: DURACION_DIAS, dtype: float64


## Análisis Exploratorio de Vigencias

In [16]:
# Análisis de distribución de duraciones
print("=" * 80)
print("DISTRIBUCIÓN DE VIGENCIAS POR DURACIÓN")
print("=" * 80)

# Categorizar vigencias
vigencias_cortas = expuestos_limpio[expuestos_limpio['DURACION_DIAS'] < 365]
vigencias_exactas = expuestos_limpio[(expuestos_limpio['DURACION_DIAS'] >= 365) & (expuestos_limpio['DURACION_DIAS'] < 366)]
vigencias_largas = expuestos_limpio[expuestos_limpio['DURACION_DIAS'] >= 365]

print(f"\n1. Vigencias < 1 año (365 días):")
print(f"   Total: {len(vigencias_cortas):,} ({100*len(vigencias_cortas)/len(expuestos_limpio):.2f}%)")
print(f"   Duración promedio: {vigencias_cortas['DURACION_DIAS'].mean():.0f} días")

print(f"\n2. Vigencias = 1 año (365-366 días):")
print(f"   Total: {len(vigencias_exactas):,} ({100*len(vigencias_exactas)/len(expuestos_limpio):.2f}%)")

print(f"\n3. Vigencias > 1 año:")
print(f"   Total: {len(vigencias_largas):,} ({100*len(vigencias_largas)/len(expuestos_limpio):.2f}%)")
print(f"   Duración promedio: {vigencias_largas['DURACION_DIAS'].mean():.0f} días ({vigencias_largas['DURACION_DIAS'].mean()/365:.2f} años)")
print(f"   Máxima duración: {vigencias_largas['DURACION_DIAS'].max():,} días ({vigencias_largas['DURACION_DIAS'].max()/365:.2f} años)")

# Mostrar ejemplos
print(f"\n" + "=" * 80)
print("EJEMPLOS DE CADA CATEGORÍA")
print("=" * 80)

print(f"\nEjemplo de vigencia corta (<1 año):")
ejemplo_corto = vigencias_cortas.iloc[0]
print(f"  Póliza: {ejemplo_corto['Poliza']}")
print(f"  Amparo: {ejemplo_corto['Amparo']}")
print(f"  Vigencia: {ejemplo_corto['VigenciaInicioPoliza']} a {ejemplo_corto['VigenciaFinPoliza']}")
print(f"  Duración: {ejemplo_corto['DURACION_DIAS']} días")
print(f"  Exposición: ${ejemplo_corto['ValorPrimaEquiv']:,.2f}")

print(f"\nEjemplo de vigencia larga (>1 año):")
ejemplo_largo = vigencias_largas.nlargest(1, 'DURACION_DIAS').iloc[0]
print(f"  Póliza: {ejemplo_largo['Poliza']}")
print(f"  Amparo: {ejemplo_largo['Amparo']}")
print(f"  Vigencia: {ejemplo_largo['VigenciaInicioPoliza']} a {ejemplo_largo['VigenciaFinPoliza']}")
print(f"  Duración: {ejemplo_largo['DURACION_DIAS']:,} días ({ejemplo_largo['DURACION_DIAS']/365:.2f} años)")
print(f"  Exposición: ${ejemplo_largo['ValorPrimaEquiv']:,.2f}")

DISTRIBUCIÓN DE VIGENCIAS POR DURACIÓN

1. Vigencias < 1 año (365 días):
   Total: 67,930 (11.18%)
   Duración promedio: 184 días

2. Vigencias = 1 año (365-366 días):
   Total: 694 (0.11%)

3. Vigencias > 1 año:
   Total: 539,477 (88.82%)
   Duración promedio: 1377 días (3.77 años)
   Máxima duración: 28,528 días (78.16 años)

EJEMPLOS DE CADA CATEGORÍA

Ejemplo de vigencia corta (<1 año):
  Póliza: 1011378
  Amparo: PRESTACIONES SOCIALES
  Vigencia: 30/06/2027 a 30/09/2027
  Duración: 93 días
  Exposición: $3,411,278.26

Ejemplo de vigencia larga (>1 año):
  Póliza: 614751
  Amparo: SERIEDAD DE LA OFERTA
  Vigencia: 10/06/2000 a 18/07/2078
  Duración: 28,528 días (78.16 años)
  Exposición: $80,000.00


## Función de Particionamiento en Períodos Anuales

In [17]:
def particionar_vigencia_en_periodos_anuales(row):
    """
    Particionar una vigencia en períodos anuales de 365 días.
    
    Retorna una lista de diccionarios, cada uno representando un período anual.
    """
    inicio = row['VIGENCIA_INICIO_DT']
    fin = row['VIGENCIA_FIN_DT']
    duracion_dias = row['DURACION_DIAS']
    
    # Caso 1: Duración inválida (fin antes de inicio o duración 0)
    if duracion_dias <= 0:
        return []  # Descartar
    
    # Caso 2: Vigencias cortas (< 365 días) - Mantener según decisión del usuario
    if duracion_dias < 365:
        return [{
            'ANIO_POLIZA': inicio.year,
            'PERIODO': 0,
            'VIGENCIA_PERIODO_INICIO': inicio,
            'VIGENCIA_PERIODO_FIN': fin,
            'DIAS_EXPOSICION': duracion_dias,
            'FRACCION_ANUAL': duracion_dias / 365.0,
            'EXPOSICION_PRORRATEADA': row['ValorPrimaEquiv'] * (duracion_dias / 365.0),
            'ES_VIGENCIA_CORTA': True
        }]
    
    # Caso 3: Vigencias >= 365 días - Particionar en períodos anuales de 365 días
    periodos = []
    fecha_inicio_periodo = inicio
    periodo_num = 0
    
    while fecha_inicio_periodo < fin:
        # Calcular fin del período: 365 días después del inicio o la fecha fin, lo que sea menor
        fecha_fin_periodo = min(fecha_inicio_periodo + pd.Timedelta(days=365), fin)
        
        # Calcular días de este período
        dias_periodo = (fecha_fin_periodo - fecha_inicio_periodo).days + 1
        fraccion = dias_periodo / 365.0
        
        periodos.append({
            'ANIO_POLIZA': fecha_inicio_periodo.year,
            'PERIODO': periodo_num,
            'VIGENCIA_PERIODO_INICIO': fecha_inicio_periodo,
            'VIGENCIA_PERIODO_FIN': fecha_fin_periodo,
            'DIAS_EXPOSICION': dias_periodo,
            'FRACCION_ANUAL': fraccion,
            'EXPOSICION_PRORRATEADA': row['ValorPrimaEquiv'] * fraccion,
            'ES_VIGENCIA_CORTA': False
        })
        
        # Avanzar al siguiente período
        fecha_inicio_periodo = fecha_fin_periodo + pd.Timedelta(days=1)
        periodo_num += 1
    
    return periodos

print("Función de particionamiento definida")
print("\nEjemplo de uso:")

# Probar con una vigencia larga
vigencia_test = expuestos_limpio[expuestos_limpio['DURACION_DIAS'] > 1095].iloc[0]
print(f"\nVigencia original:")
print(f"  Póliza: {vigencia_test['Poliza']}")
print(f"  Vigencia: {vigencia_test['VigenciaInicioPoliza']} a {vigencia_test['VigenciaFinPoliza']}")
print(f"  Duración: {vigencia_test['DURACION_DIAS']:,} días ({vigencia_test['DURACION_DIAS']/365:.2f} años)")
print(f"  Exposición: ${vigencia_test['ValorPrimaEquiv']:,.2f}")

periodos_test = particionar_vigencia_en_periodos_anuales(vigencia_test)
print(f"\nPeríodos generados: {len(periodos_test)}")
for i, periodo in enumerate(periodos_test[:5]):  # Mostrar solo los primeros 5
    print(f"\n  Período {i}:")
    print(f"    Año: {periodo['ANIO_POLIZA']}")
    print(f"    Vigencia: {periodo['VIGENCIA_PERIODO_INICIO'].strftime('%Y-%m-%d')} a {periodo['VIGENCIA_PERIODO_FIN'].strftime('%Y-%m-%d')}")
    print(f"    Días: {periodo['DIAS_EXPOSICION']}")
    print(f"    Fracción anual: {periodo['FRACCION_ANUAL']:.4f}")
    print(f"    Exposición prorrateada: ${periodo['EXPOSICION_PRORRATEADA']:,.2f}")

if len(periodos_test) > 5:
    print(f"\n  ... y {len(periodos_test) - 5} períodos más")

Función de particionamiento definida

Ejemplo de uso:

Vigencia original:
  Póliza: 62512
  Vigencia: 09/07/2003 a 09/10/2030
  Duración: 9,955 días (27.27 años)
  Exposición: $11,544.00

Períodos generados: 28

  Período 0:
    Año: 2003
    Vigencia: 2003-07-09 a 2004-07-08
    Días: 366
    Fracción anual: 1.0027
    Exposición prorrateada: $11,575.63

  Período 1:
    Año: 2004
    Vigencia: 2004-07-09 a 2005-07-09
    Días: 366
    Fracción anual: 1.0027
    Exposición prorrateada: $11,575.63

  Período 2:
    Año: 2005
    Vigencia: 2005-07-10 a 2006-07-10
    Días: 366
    Fracción anual: 1.0027
    Exposición prorrateada: $11,575.63

  Período 3:
    Año: 2006
    Vigencia: 2006-07-11 a 2007-07-11
    Días: 366
    Fracción anual: 1.0027
    Exposición prorrateada: $11,575.63

  Período 4:
    Año: 2007
    Vigencia: 2007-07-12 a 2008-07-11
    Días: 366
    Fracción anual: 1.0027
    Exposición prorrateada: $11,575.63

  ... y 23 períodos más


## Aplicar Particionamiento a Todos los Expuestos

In [18]:
# Aplicar particionamiento a todos los registros
print("=" * 80)
print("PARTICIONANDO VIGENCIAS EN PERÍODOS ANUALES")
print("=" * 80)
print(f"\nRegistros originales: {len(expuestos_limpio):,}")
print("Procesando... esto puede tomar unos momentos\n")

expuestos_expandidos = []
vigencias_descartadas = 0

# Procesar cada fila
for idx, row in expuestos_limpio.iterrows():
    if idx % 100000 == 0 and idx > 0:
        print(f"  Procesados: {idx:,} / {len(expuestos_limpio):,}")
    
    periodos = particionar_vigencia_en_periodos_anuales(row)
    
    if len(periodos) == 0:
        vigencias_descartadas += 1
        continue
    
    # Agregar información original del row a cada período
    for periodo in periodos:
        registro_expandido = {
            # Información de la vigencia original (para trazabilidad)
            'VIGENCIA_ORIGINAL_ID': f"{row['Poliza']}_{row['Amparo']}_{row['VigenciaInicioPoliza']}",
            'POLIZA': row['Poliza'],
            'AMPARO': row['Amparo'],
            'COD_SUCURSAL': row['CodSucursal'],
            'VIGENCIA_ORIGINAL_INICIO': row['VigenciaInicioPoliza'],
            'VIGENCIA_ORIGINAL_FIN': row['VigenciaFinPoliza'],
            'EXPOSICION_ORIGINAL': row['ValorPrimaEquiv'],
            # Información del período anual
            **periodo
        }
        expuestos_expandidos.append(registro_expandido)

print(f"  Procesados: {len(expuestos_limpio):,} / {len(expuestos_limpio):,}")

# Crear dataframe de períodos
expuestos_periodos = pd.DataFrame(expuestos_expandidos)

print(f"\n" + "=" * 80)
print("RESULTADOS DEL PARTICIONAMIENTO")
print("=" * 80)
print(f"\nVigencias originales: {len(expuestos_limpio):,}")
print(f"Vigencias descartadas (duración inválida): {vigencias_descartadas:,}")
print(f"Períodos anuales generados: {len(expuestos_periodos):,}")
print(f"Factor de expansión: {len(expuestos_periodos)/len(expuestos_limpio):.2f}x")

print(f"\nDistribución de períodos:")
print(f"  Vigencias cortas (< 1 año): {expuestos_periodos['ES_VIGENCIA_CORTA'].sum():,}")
print(f"  Períodos de vigencias largas: {(~expuestos_periodos['ES_VIGENCIA_CORTA']).sum():,}")

print(f"\nPrimeras filas del dataframe de períodos:")
expuestos_periodos.head()

PARTICIONANDO VIGENCIAS EN PERÍODOS ANUALES

Registros originales: 607,407
Procesando... esto puede tomar unos momentos

  Procesados: 100,000 / 607,407
  Procesados: 200,000 / 607,407
  Procesados: 300,000 / 607,407
  Procesados: 400,000 / 607,407
  Procesados: 500,000 / 607,407
  Procesados: 600,000 / 607,407
  Procesados: 607,407 / 607,407

RESULTADOS DEL PARTICIONAMIENTO

Vigencias originales: 607,407
Vigencias descartadas (duración inválida): 0
Períodos anuales generados: 2,411,586
Factor de expansión: 3.97x

Distribución de períodos:
  Vigencias cortas (< 1 año): 67,930
  Períodos de vigencias largas: 2,343,656

Primeras filas del dataframe de períodos:


,VIGENCIA_ORIGINAL_ID,POLIZA,AMPARO,COD_SUCURSAL,VIGENCIA_ORIGINAL_INICIO,VIGENCIA_ORIGINAL_FIN,EXPOSICION_ORIGINAL,ANIO_POLIZA,PERIODO,VIGENCIA_PERIODO_INICIO,VIGENCIA_PERIODO_FIN,DIAS_EXPOSICION,FRACCION_ANUAL,EXPOSICION_PRORRATEADA,ES_VIGENCIA_CORTA
0,62512_ESTABILIDAD DE LA OBRA_09/07/2003,62512,ESTABILIDAD DE LA OBRA,9,09/07/2003,09/10/2030,11544.0,2003,0,2003-07-09,2004-07-08,366,1.00274,11575.627397,False
1,62512_ESTABILIDAD DE LA OBRA_09/07/2003,62512,ESTABILIDAD DE LA OBRA,9,09/07/2003,09/10/2030,11544.0,2004,1,2004-07-09,2005-07-09,366,1.00274,11575.627397,False
2,62512_ESTABILIDAD DE LA OBRA_09/07/2003,62512,ESTABILIDAD DE LA OBRA,9,09/07/2003,09/10/2030,11544.0,2005,2,2005-07-10,2006-07-10,366,1.00274,11575.627397,False
3,62512_ESTABILIDAD DE LA OBRA_09/07/2003,62512,ESTABILIDAD DE LA OBRA,9,09/07/2003,09/10/2030,11544.0,2006,3,2006-07-11,2007-07-11,366,1.00274,11575.627397,False
4,62512_ESTABILIDAD DE LA OBRA_09/07/2003,62512,ESTABILIDAD DE LA OBRA,9,09/07/2003,09/10/2030,11544.0,2007,4,2007-07-12,2008-07-11,366,1.00274,11575.627397,False


In [19]:
# =============================================================================
# RESUMEN DE PERÍODOS ANUALES GENERADOS
# =============================================================================
print("=" * 80)
print("RESUMEN DE PERÍODOS ANUALES GENERADOS")
print("=" * 80)

print(f"\nTotal de períodos generados: {len(expuestos_periodos):,}")

# Mostrar rango de fechas
print(f"\nRango de fechas de períodos:")
print(f"  Inicio mínimo: {expuestos_periodos['VIGENCIA_PERIODO_INICIO'].min()}")
print(f"  Inicio máximo: {expuestos_periodos['VIGENCIA_PERIODO_INICIO'].max()}")
print(f"  Fin mínimo: {expuestos_periodos['VIGENCIA_PERIODO_FIN'].min()}")
print(f"  Fin máximo: {expuestos_periodos['VIGENCIA_PERIODO_FIN'].max()}")

# Estadísticas por año de inicio
print(f"\nDistribución por año de inicio de período:")
print(expuestos_periodos['ANIO_POLIZA'].value_counts().sort_index())

# Nota: El filtro temporal se aplicará DESPUÉS del cruce con siniestros

RESUMEN DE PERÍODOS ANUALES GENERADOS

Total de períodos generados: 2,411,586

Rango de fechas de períodos:
  Inicio mínimo: 2000-06-10 00:00:00
  Inicio máximo: 2077-08-07 00:00:00
  Fin mínimo: 2001-06-10 00:00:00
  Fin máximo: 2078-07-18 00:00:00

Distribución por año de inicio de período:
ANIO_POLIZA
2000    1
2001    1
2002    1
2003    2
2004    2
       ..
2073    1
2074    1
2075    1
2076    1
2077    1
Name: count, Length: 78, dtype: int64


## Validación: Conservación de Exposición

In [20]:
# Validar que la suma de exposiciones prorrateadas ≈ exposición original
print("=" * 80)
print("VALIDACIÓN: CONSERVACIÓN DE EXPOSICIÓN")
print("=" * 80)

# Agrupar por vigencia original y sumar exposiciones prorrateadas
validacion = expuestos_periodos.groupby('VIGENCIA_ORIGINAL_ID').agg({
    'EXPOSICION_PRORRATEADA': 'sum',
    'EXPOSICION_ORIGINAL': 'first'
}).reset_index()

validacion['DIFERENCIA'] = validacion['EXPOSICION_PRORRATEADA'] - validacion['EXPOSICION_ORIGINAL']
validacion['DIFERENCIA_PCT'] = (validacion['DIFERENCIA'] / validacion['EXPOSICION_ORIGINAL']) * 100

print(f"\nEstadísticas de conservación de exposición:")
print(f"  Diferencia promedio: ${validacion['DIFERENCIA'].mean():,.2f}")
print(f"  Diferencia máxima: ${validacion['DIFERENCIA'].abs().max():,.2f}")
print(f"  Diferencia % promedio: {validacion['DIFERENCIA_PCT'].mean():.6f}%")
print(f"  Diferencia % máxima: {validacion['DIFERENCIA_PCT'].abs().max():.6f}%")

# Verificar si la conservación es adecuada (< 1% de diferencia)
if validacion['DIFERENCIA_PCT'].abs().max() < 1.0:
    print(f"\n✅ VALIDACIÓN EXITOSA: La exposición se conserva correctamente (diferencia < 1%)")
else:
    print(f"\n⚠️  ADVERTENCIA: Hay diferencias significativas en la conservación de exposición")

VALIDACIÓN: CONSERVACIÓN DE EXPOSICIÓN

Estadísticas de conservación de exposición:
  Diferencia promedio: $1,981,991,770.99
  Diferencia máxima: $55,650,343,932,011.79
  Diferencia % promedio: 241.425206%
  Diferencia % máxima: 7715.890411%

⚠️  ADVERTENCIA: Hay diferencias significativas en la conservación de exposición


In [21]:
# Crear ID único para cada período: POLIZA_AMPARO_AÑO_PERIODO_YYYYMMDD
expuestos_periodos['ID_POLIZA_AMPARO_PERIODO'] = (
    expuestos_periodos['POLIZA'].astype(str) + '_' +
    expuestos_periodos['AMPARO'] + '_' +
    expuestos_periodos['ANIO_POLIZA'].astype(str) + '_' +
    expuestos_periodos['PERIODO'].astype(str) + '_' +
    expuestos_periodos['VIGENCIA_PERIODO_INICIO'].dt.strftime('%Y%m%d')
)

print(f"IDs únicos creados: {expuestos_periodos['ID_POLIZA_AMPARO_PERIODO'].nunique():,}")
print(f"Total de períodos: {len(expuestos_periodos):,}")
print(f"\nPrimeros 5 IDs:")
for id_ejemplo in expuestos_periodos['ID_POLIZA_AMPARO_PERIODO'].head():
    print(f"  - {id_ejemplo}")

IDs únicos creados: 2,411,586
Total de períodos: 2,411,586

Primeros 5 IDs:
  - 62512_ESTABILIDAD DE LA OBRA_2003_0_20030709
  - 62512_ESTABILIDAD DE LA OBRA_2004_1_20040709
  - 62512_ESTABILIDAD DE LA OBRA_2005_2_20050710
  - 62512_ESTABILIDAD DE LA OBRA_2006_3_20060711
  - 62512_ESTABILIDAD DE LA OBRA_2007_4_20070712


## Preparar Fechas de Siniestros para JOIN Temporal

In [22]:
# Convertir fecha de siniestro a datetime
siniestros_consolidado['FECHA_SINIESTRO_DT'] = pd.to_datetime(
    siniestros_consolidado['FECHA_DE_SINIESTRO'],
    format='%d/%m/%Y',
    errors='coerce'
)

print(f"Fechas de siniestro convertidas:")
print(f"  Total siniestros: {len(siniestros_consolidado):,}")
print(f"  Con fecha válida: {siniestros_consolidado['FECHA_SINIESTRO_DT'].notna().sum():,}")
print(f"  Con fecha inválida: {siniestros_consolidado['FECHA_SINIESTRO_DT'].isna().sum():,}")

Fechas de siniestro convertidas:
  Total siniestros: 5,746
  Con fecha válida: 5,746
  Con fecha inválida: 0


## JOIN Temporal: Asignar Siniestros a Períodos Anuales

In [23]:
print("=" * 80)
print("JOIN TEMPORAL: ASIGNANDO SINIESTROS A PERÍODOS (LÓGICA MEJORADA)")
print("=" * 80)

# =============================================================================
# PASO 1: Merge inicial por POLIZA-AMPARO
# =============================================================================
print("\nPaso 1: Merge inicial por POLIZA-AMPARO...")
merge_inicial = expuestos_periodos.merge(
    siniestros_consolidado[['POLIZA', 'AMPARO', 'FECHA_SINIESTRO_DT', 
                            'DEPARTAMENTO_SINIESTRO', 'FECHA_DE_SINIESTRO', 
                            'FECHA_AVISO', 'PAGOS', 'RESERVA_ACTUAL_EQUI', 'VALOR_ASEGURADO']],
    on=['POLIZA', 'AMPARO'],
    how='inner'
)
print(f"  Combinaciones POLIZA-AMPARO encontradas: {len(merge_inicial):,}")

# =============================================================================
# PASO 2: Identificar el PERÍODO ORIGINAL donde ocurrió el siniestro
# =============================================================================
print("\nPaso 2: Identificando períodos originales del siniestro...")

# Períodos donde el siniestro ocurrió EXACTAMENTE (para asignar severidad)
periodos_originales = merge_inicial[
    (merge_inicial['VIGENCIA_PERIODO_INICIO'] <= merge_inicial['FECHA_SINIESTRO_DT']) &
    (merge_inicial['VIGENCIA_PERIODO_FIN'] >= merge_inicial['FECHA_SINIESTRO_DT'])
].copy()

print(f"  Períodos originales con siniestro: {len(periodos_originales):,}")

# =============================================================================
# PASO 3: Identificar TODOS los períodos afectados (actuales + posteriores)
# =============================================================================
print("\nPaso 3: Identificando períodos afectados (original + posteriores)...")

# NUEVA LÓGICA: El siniestro afecta si ocurrió ANTES O DURANTE el período
# Es decir: FECHA_SINIESTRO_DT <= VIGENCIA_PERIODO_FIN
periodos_afectados = merge_inicial[
    merge_inicial['FECHA_SINIESTRO_DT'] <= merge_inicial['VIGENCIA_PERIODO_FIN']
].copy()

# Inicializar columna ES_PERIODO_ORIGINAL
periodos_afectados['ES_PERIODO_ORIGINAL'] = False

# Marcar los períodos originales
ids_originales = set(periodos_originales['ID_POLIZA_AMPARO_PERIODO'].unique())
periodos_afectados.loc[
    periodos_afectados['ID_POLIZA_AMPARO_PERIODO'].isin(ids_originales),
    'ES_PERIODO_ORIGINAL'
] = True

print(f"  Períodos totales afectados: {len(periodos_afectados):,}")
print(f"    - Períodos originales (con severidad): {periodos_afectados['ES_PERIODO_ORIGINAL'].sum():,}")
print(f"    - Períodos posteriores (heredados): {(~periodos_afectados['ES_PERIODO_ORIGINAL']).sum():,}")

# =============================================================================
# PASO 4: Identificar siniestros no asignados
# =============================================================================
print("\nPaso 4: Identificando siniestros no asignados...")
polizas_amparo_asignados = periodos_afectados[['POLIZA', 'AMPARO']].drop_duplicates()
siniestros_no_asignados_df = siniestros_consolidado.merge(
    polizas_amparo_asignados,
    on=['POLIZA', 'AMPARO'],
    how='left',
    indicator=True
)
siniestros_no_asignados_df = siniestros_no_asignados_df[
    siniestros_no_asignados_df['_merge'] == 'left_only'
].drop(columns=['_merge'])
print(f"  Siniestros no asignados: {len(siniestros_no_asignados_df):,}")

# =============================================================================
# PASO 5: Identificar períodos SIN siniestro
# =============================================================================
print("\nPaso 5: Identificando períodos sin siniestro...")
ids_con_siniestro = set(periodos_afectados['ID_POLIZA_AMPARO_PERIODO'].unique())
periodos_sin_siniestro_df = expuestos_periodos[
    ~expuestos_periodos['ID_POLIZA_AMPARO_PERIODO'].isin(ids_con_siniestro)
].copy()

# Agregar columnas de siniestro con valores nulos/cero
periodos_sin_siniestro_df['DEPARTAMENTO_SINIESTRO'] = None
periodos_sin_siniestro_df['FECHA_DE_SINIESTRO'] = None
periodos_sin_siniestro_df['FECHA_AVISO'] = None
periodos_sin_siniestro_df['PAGOS'] = 0.0
periodos_sin_siniestro_df['RESERVA_ACTUAL_EQUI'] = 0.0
periodos_sin_siniestro_df['VALOR_ASEGURADO'] = None
periodos_sin_siniestro_df['FECHA_SINIESTRO_DT'] = None
periodos_sin_siniestro_df['ES_PERIODO_ORIGINAL'] = False

print(f"  Períodos sin siniestro: {len(periodos_sin_siniestro_df):,}")

# =============================================================================
# PASO 6: Combinar todos los períodos
# =============================================================================
print("\nPaso 6: Combinando base final...")
base_completa = pd.concat([periodos_afectados, periodos_sin_siniestro_df], ignore_index=True)

# Convertir a lista para compatibilidad
siniestros_no_asignados = siniestros_no_asignados_df.to_dict('records')

print(f"\n" + "=" * 80)
print("RESULTADOS DEL JOIN TEMPORAL (LÓGICA MEJORADA)")
print("=" * 80)
print(f"\nSiniestros procesados: {len(siniestros_consolidado):,}")
print(f"Siniestros asignados a períodos: {len(siniestros_consolidado) - len(siniestros_no_asignados):,}")
print(f"Siniestros NO asignados: {len(siniestros_no_asignados):,}")
print(f"Tasa de asignación: {((len(siniestros_consolidado) - len(siniestros_no_asignados)) / len(siniestros_consolidado) * 100):.2f}%")
print(f"\nPeríodos totales en base completa: {len(base_completa):,}")
print(f"  Con siniestro (original): {periodos_afectados['ES_PERIODO_ORIGINAL'].sum():,}")
print(f"  Con siniestro (heredado): {(~periodos_afectados['ES_PERIODO_ORIGINAL']).sum():,}")
print(f"  Sin siniestro: {len(periodos_sin_siniestro_df):,}")

JOIN TEMPORAL: ASIGNANDO SINIESTROS A PERÍODOS (LÓGICA MEJORADA)

Paso 1: Merge inicial por POLIZA-AMPARO...
  Combinaciones POLIZA-AMPARO encontradas: 10,374

Paso 2: Identificando períodos originales del siniestro...
  Períodos originales con siniestro: 452

Paso 3: Identificando períodos afectados (original + posteriores)...
  Períodos totales afectados: 10,111
    - Períodos originales (con severidad): 452
    - Períodos posteriores (heredados): 9,659

Paso 4: Identificando siniestros no asignados...
  Siniestros no asignados: 3,162

Paso 5: Identificando períodos sin siniestro...
  Períodos sin siniestro: 2,401,475

Paso 6: Combinando base final...


/tmp/ipykernel_339244/3279653435.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  base_completa = pd.concat([periodos_afectados, periodos_sin_siniestro_df], ignore_index=True)



RESULTADOS DEL JOIN TEMPORAL (LÓGICA MEJORADA)

Siniestros procesados: 5,746
Siniestros asignados a períodos: 2,584
Siniestros NO asignados: 3,162
Tasa de asignación: 44.97%

Períodos totales en base completa: 2,411,586
  Con siniestro (original): 452
  Con siniestro (heredado): 9,659
  Sin siniestro: 2,401,475


## Filtro Temporal: Períodos Completamente Transcurridos

In [24]:
# =============================================================================
# FILTRO TEMPORAL: PERÍODOS COMPLETAMENTE TRANSCURRIDOS
# =============================================================================
print("=" * 80)
print("FILTRO TEMPORAL: PERÍODOS COMPLETAMENTE TRANSCURRIDOS")
print("=" * 80)

# Fecha de corte: solo períodos que ya terminaron
fecha_corte = pd.Timestamp('2025-12-07')

print(f"\nAntes del filtro: {len(base_completa):,} períodos")

# Filtrar: solo períodos cuya vigencia haya terminado antes de la fecha de corte
base_completa = base_completa[
    base_completa['VIGENCIA_PERIODO_FIN'] <= fecha_corte
].copy()

print(f"Después del filtro: {len(base_completa):,} períodos")

# Mostrar rango de fechas resultante
print(f"\nRango de fechas de períodos (post-filtro):")
print(f"  Inicio mínimo: {base_completa['VIGENCIA_PERIODO_INICIO'].min()}")
print(f"  Inicio máximo: {base_completa['VIGENCIA_PERIODO_INICIO'].max()}")
print(f"  Fin mínimo: {base_completa['VIGENCIA_PERIODO_FIN'].min()}")
print(f"  Fin máximo: {base_completa['VIGENCIA_PERIODO_FIN'].max()}")

# Distribución por año
print(f"\nDistribución por año de inicio de período:")
print(base_completa['ANIO_POLIZA'].value_counts().sort_index())

FILTRO TEMPORAL: PERÍODOS COMPLETAMENTE TRANSCURRIDOS

Antes del filtro: 2,411,586 períodos
Después del filtro: 740,033 períodos

Rango de fechas de períodos (post-filtro):
  Inicio mínimo: 2000-06-10 00:00:00
  Inicio máximo: 2025-12-05 00:00:00
  Fin mínimo: 2001-06-10 00:00:00
  Fin máximo: 2025-12-07 00:00:00

Distribución por año de inicio de período:
ANIO_POLIZA
2000         1
2001         1
2002         1
2003         2
2004         2
2005         2
2006         2
2007         3
2008         6
2009         7
2010         7
2011         7
2012         8
2013         8
2014        10
2015        15
2016        28
2017        67
2018       155
2019       644
2020      5858
2021     20375
2022     98586
2023    204573
2024    349355
2025     60310
Name: count, dtype: int64


## Validación de Asignación de Siniestros

In [25]:
print("=" * 80)
print("VALIDACIÓN DE ASIGNACIÓN DE SINIESTROS")
print("=" * 80)

print(f"\nTotal de siniestros: {len(siniestros_consolidado):,}")
print(f"Asignados: {len(siniestros_consolidado) - len(siniestros_no_asignados):,}")
print(f"No asignados: {len(siniestros_no_asignados):,}")
print(f"Tasa de asignación: {((len(siniestros_consolidado) - len(siniestros_no_asignados)) / len(siniestros_consolidado) * 100):.2f}%")

if len(siniestros_no_asignados) > 0:
    print(f"\nPrimeros 5 siniestros no asignados:")
    for i, sin in enumerate(siniestros_no_asignados_df.head()['POLIZA']):
        print(f"  {i+1}. Póliza: {sin}")

VALIDACIÓN DE ASIGNACIÓN DE SINIESTROS

Total de siniestros: 5,746
Asignados: 2,584
No asignados: 3,162
Tasa de asignación: 44.97%

Primeros 5 siniestros no asignados:
  1. Póliza: 1450
  2. Póliza: 6232
  3. Póliza: 6296
  4. Póliza: 10309
  5. Póliza: 14213


## Filtrar Exposición Inválida

In [26]:
# Filtrar registros con exposición válida
print(f"Registros antes de filtrar: {len(base_completa):,}")

base_filtrada = base_completa[
    (base_completa['EXPOSICION_PRORRATEADA'].notna()) &
    (base_completa['EXPOSICION_PRORRATEADA'] > 0)
].copy()

print(f"Registros después de filtrar: {len(base_filtrada):,}")
print(f"Registros eliminados: {len(base_completa) - len(base_filtrada):,}")

Registros antes de filtrar: 740,033
Registros después de filtrar: 738,902
Registros eliminados: 1,131


## Calcular Severidad Acotada

In [27]:
import numpy as np

# =============================================================================
# CÁLCULO DE SEVERIDAD (DIFERENCIANDO ORIGINAL vs HEREDADO)
# =============================================================================
print("=" * 80)
print("CÁLCULO DE SEVERIDAD (DIFERENCIANDO ORIGINAL vs HEREDADO)")
print("=" * 80)

# Para períodos ORIGINALES: SEVERIDAD = min(max(RESERVA, PAGOS), EXPOSICION)
# Para períodos HEREDADOS: SEVERIDAD = 0 (pero TUVO_SINIESTRO = 1)

# Inicializar severidad en 0
base_filtrada['SEVERIDAD'] = 0.0

# Calcular severidad SOLO para períodos originales
mask_original = base_filtrada['ES_PERIODO_ORIGINAL'] == True

if mask_original.sum() > 0:
    base_filtrada.loc[mask_original, 'SEVERIDAD'] = np.minimum(
        np.maximum(
            base_filtrada.loc[mask_original, 'RESERVA_ACTUAL_EQUI'].fillna(0),
            base_filtrada.loc[mask_original, 'PAGOS'].fillna(0)
        ),
        base_filtrada.loc[mask_original, 'EXPOSICION_PRORRATEADA']
    )

print(f"\nSeveridad calculada:")
print(f"  Períodos con severidad > 0: {(base_filtrada['SEVERIDAD'] > 0).sum():,}")
print(f"  Períodos con severidad = 0: {(base_filtrada['SEVERIDAD'] == 0).sum():,}")

print(f"\nDesglose de períodos con SEVERIDAD = 0:")
print(f"  - Sin siniestro: {(~base_filtrada['FECHA_SINIESTRO_DT'].notna()).sum():,}")
print(f"  - Heredados (siniestro anterior): {((base_filtrada['FECHA_SINIESTRO_DT'].notna()) & (base_filtrada['SEVERIDAD'] == 0)).sum():,}")

if (base_filtrada['SEVERIDAD'] > 0).sum() > 0:
    print(f"\nEstadísticas de SEVERIDAD (solo donde > 0):")
    print(base_filtrada[base_filtrada['SEVERIDAD'] > 0]['SEVERIDAD'].describe())

CÁLCULO DE SEVERIDAD (DIFERENCIANDO ORIGINAL vs HEREDADO)

Severidad calculada:
  Períodos con severidad > 0: 404
  Períodos con severidad = 0: 738,498

Desglose de períodos con SEVERIDAD = 0:
  - Sin siniestro: 736,552
  - Heredados (siniestro anterior): 1,946

Estadísticas de SEVERIDAD (solo donde > 0):
count    4.040000e+02
mean     2.366607e+07
std      7.802636e+07
min      1.075342e+04
25%      6.689301e+05
50%      2.665911e+06
75%      1.423962e+07
max      1.220499e+09
Name: SEVERIDAD, dtype: float64


## Extrapolación Lineal a Base Anual (INNOVACIÓN)

In [28]:
print("=" * 80)
print("EXTRAPOLACIÓN LINEAL A BASE ANUAL")
print("=" * 80)

# Extrapolar severidad a base anual para comparabilidad
base_filtrada['SEVERIDAD_EXTRAPOLADA_ANUAL'] = (
    base_filtrada['SEVERIDAD'] * (365.0 / base_filtrada['DIAS_EXPOSICION'])
)

print(f"\nSeveridad extrapolada calculada")
print(f"\nPeríodos con vigencia < 1 año y siniestro:")
periodos_cortos_siniestrados = base_filtrada[
    (base_filtrada['DIAS_EXPOSICION'] < 365) &
    (base_filtrada['SEVERIDAD'] > 0)
]
print(f"  Total: {len(periodos_cortos_siniestrados):,}")
if len(periodos_cortos_siniestrados) > 0:
    print(f"  Factor extrapolación promedio: {(365.0 / periodos_cortos_siniestrados['DIAS_EXPOSICION']).mean():.2f}x")
    print(f"  Severidad promedio original: ${periodos_cortos_siniestrados['SEVERIDAD'].mean():,.2f}")
    print(f"  Severidad promedio extrapolada: ${periodos_cortos_siniestrados['SEVERIDAD_EXTRAPOLADA_ANUAL'].mean():,.2f}")

print(f"\nEstadísticas de SEVERIDAD_EXTRAPOLADA_ANUAL:")
print(base_filtrada['SEVERIDAD_EXTRAPOLADA_ANUAL'].describe())

EXTRAPOLACIÓN LINEAL A BASE ANUAL

Severidad extrapolada calculada

Períodos con vigencia < 1 año y siniestro:
  Total: 5
  Factor extrapolación promedio: 1.73x
  Severidad promedio original: $49,452.80
  Severidad promedio extrapolada: $72,835.90

Estadísticas de SEVERIDAD_EXTRAPOLADA_ANUAL:
count    7.389020e+05
mean     1.290440e+04
std      1.899146e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.217164e+09
Name: SEVERIDAD_EXTRAPOLADA_ANUAL, dtype: float64


## Crear Indicador TUVO_SINIESTRO

In [29]:
# =============================================================================
# CREAR INDICADOR TUVO_SINIESTRO
# =============================================================================
# TUVO_SINIESTRO = 1 si:
#   - Es período original del siniestro (tiene severidad), O
#   - Es período heredado (posterior al siniestro, severidad = 0)

# Verificar si hay siniestro asignado (fecha no nula)
base_filtrada['TUVO_SINIESTRO'] = base_filtrada['FECHA_SINIESTRO_DT'].notna().astype(int)

print("Indicador TUVO_SINIESTRO creado")
print(f"\nDistribución:")
print(base_filtrada['TUVO_SINIESTRO'].value_counts())
print(f"\nTasa de siniestralidad: {base_filtrada['TUVO_SINIESTRO'].mean():.2%}")

# Desglose por tipo de período
print(f"\nDesglose de períodos con siniestro:")
periodos_con_siniestro = base_filtrada[base_filtrada['TUVO_SINIESTRO'] == 1]
print(f"  Total con TUVO_SINIESTRO=1: {len(periodos_con_siniestro):,}")
print(f"    - Períodos originales (SEVERIDAD > 0): {(periodos_con_siniestro['SEVERIDAD'] > 0).sum():,}")
print(f"    - Períodos heredados (SEVERIDAD = 0): {(periodos_con_siniestro['SEVERIDAD'] == 0).sum():,}")

Indicador TUVO_SINIESTRO creado

Distribución:
TUVO_SINIESTRO
0    736552
1      2350
Name: count, dtype: int64

Tasa de siniestralidad: 0.32%

Desglose de períodos con siniestro:
  Total con TUVO_SINIESTRO=1: 2,350
    - Períodos originales (SEVERIDAD > 0): 404
    - Períodos heredados (SEVERIDAD = 0): 1,946


## Selección de Columnas Finales

In [30]:
# Seleccionar columnas finales
base_final = base_filtrada[[
    'ID_POLIZA_AMPARO_PERIODO',
    'POLIZA',
    'AMPARO',
    'ANIO_POLIZA',
    'PERIODO',
    'VIGENCIA_PERIODO_INICIO',
    'VIGENCIA_PERIODO_FIN',
    'DIAS_EXPOSICION',
    'FRACCION_ANUAL',
    'EXPOSICION_PRORRATEADA',
    'VIGENCIA_ORIGINAL_ID',
    'COD_SUCURSAL',
    'VIGENCIA_ORIGINAL_INICIO',
    'VIGENCIA_ORIGINAL_FIN',
    'DEPARTAMENTO_SINIESTRO',
    'FECHA_DE_SINIESTRO',
    'FECHA_AVISO',
    'SEVERIDAD',
    'SEVERIDAD_EXTRAPOLADA_ANUAL',
    'TUVO_SINIESTRO',
    'ES_PERIODO_ORIGINAL'  # NUEVA COLUMNA
]].copy()

print(f"Columnas finales seleccionadas: {len(base_final.columns)}")
print(f"\nColumnas:")
for i, col in enumerate(base_final.columns, 1):
    print(f"  {i}. {col}")

Columnas finales seleccionadas: 21

Columnas:
  1. ID_POLIZA_AMPARO_PERIODO
  2. POLIZA
  3. AMPARO
  4. ANIO_POLIZA
  5. PERIODO
  6. VIGENCIA_PERIODO_INICIO
  7. VIGENCIA_PERIODO_FIN
  8. DIAS_EXPOSICION
  9. FRACCION_ANUAL
  10. EXPOSICION_PRORRATEADA
  11. VIGENCIA_ORIGINAL_ID
  12. COD_SUCURSAL
  13. VIGENCIA_ORIGINAL_INICIO
  14. VIGENCIA_ORIGINAL_FIN
  15. DEPARTAMENTO_SINIESTRO
  16. FECHA_DE_SINIESTRO
  17. FECHA_AVISO
  18. SEVERIDAD
  19. SEVERIDAD_EXTRAPOLADA_ANUAL
  20. TUVO_SINIESTRO
  21. ES_PERIODO_ORIGINAL


## Validaciones Finales

In [31]:
print("=" * 80)
print("VALIDACIONES FINALES")
print("=" * 80)

print(f"\n1. IDs únicos:")
print(f"   Total de registros: {len(base_final):,}")
print(f"   IDs únicos: {base_final['ID_POLIZA_AMPARO_PERIODO'].nunique():,}")
print(f"   ¿Hay duplicados?: {'SÍ' if base_final['ID_POLIZA_AMPARO_PERIODO'].nunique() < len(base_final) else 'NO'}")

print(f"\n2. Valores nulos en columnas críticas:")
print(f"   POLIZA: {base_final['POLIZA'].isna().sum():,}")
print(f"   AMPARO: {base_final['AMPARO'].isna().sum():,}")
print(f"   EXPOSICION_PRORRATEADA: {base_final['EXPOSICION_PRORRATEADA'].isna().sum():,}")
print(f"   SEVERIDAD: {base_final['SEVERIDAD'].isna().sum():,}")
print(f"   SEVERIDAD_EXTRAPOLADA_ANUAL: {base_final['SEVERIDAD_EXTRAPOLADA_ANUAL'].isna().sum():,}")

print(f"\n3. Rangos de valores:")
print(f"   Exposición prorrateada > 0: {(base_final['EXPOSICION_PRORRATEADA'] > 0).all()}")
print(f"   Severidad >= 0: {(base_final['SEVERIDAD'] >= 0).all()}")
print(f"   Severidad extrapolada >= 0: {(base_final['SEVERIDAD_EXTRAPOLADA_ANUAL'] >= 0).all()}")

VALIDACIONES FINALES

1. IDs únicos:
   Total de registros: 738,902
   IDs únicos: 738,902
   ¿Hay duplicados?: NO

2. Valores nulos en columnas críticas:
   POLIZA: 0
   AMPARO: 0
   EXPOSICION_PRORRATEADA: 0
   SEVERIDAD: 0
   SEVERIDAD_EXTRAPOLADA_ANUAL: 0

3. Rangos de valores:
   Exposición prorrateada > 0: True
   Severidad >= 0: True
   Severidad extrapolada >= 0: True


## Resumen Estadístico Final

In [32]:
print("=" * 80)
print("RESUMEN ESTADÍSTICO FINAL")
print("=" * 80)

print(f"\n1. DIMENSIONES")
print(f"   Registros totales: {len(base_final):,}")
print(f"   Pólizas únicas: {base_final['POLIZA'].nunique():,}")
print(f"   Combinaciones POLIZA-AMPARO únicas: {base_final.groupby(['POLIZA', 'AMPARO']).ngroups:,}")

print(f"\n2. DISTRIBUCIÓN TEMPORAL")
print(f"   Años únicos: {base_final['ANIO_POLIZA'].nunique()}")
print(f"   Rango de años: {base_final['ANIO_POLIZA'].min()} - {base_final['ANIO_POLIZA'].max()}")

print(f"\n3. EXPOSICIÓN")
print(f"   Total: ${base_final['EXPOSICION_PRORRATEADA'].sum():,.2f}")
print(f"   Promedio: ${base_final['EXPOSICION_PRORRATEADA'].mean():,.2f}")
print(f"   Mediana: ${base_final['EXPOSICION_PRORRATEADA'].median():,.2f}")

print(f"\n4. SINIESTRALIDAD")
print(f"   Registros con siniestro: {base_final['TUVO_SINIESTRO'].sum():,}")
print(f"   Registros sin siniestro: {(1 - base_final['TUVO_SINIESTRO']).sum():,}")
print(f"   Tasa de siniestralidad: {base_final['TUVO_SINIESTRO'].mean():.2%}")

print(f"\n5. SEVERIDAD (solo siniestrados)")
siniestrados = base_final[base_final['TUVO_SINIESTRO'] == 1]
if len(siniestrados) > 0:
    print(f"   Severidad promedio: ${siniestrados['SEVERIDAD'].mean():,.2f}")
    print(f"   Severidad mediana: ${siniestrados['SEVERIDAD'].median():,.2f}")
    print(f"   Severidad máxima: ${siniestrados['SEVERIDAD'].max():,.2f}")
    print(f"\n   Severidad extrapolada promedio: ${siniestrados['SEVERIDAD_EXTRAPOLADA_ANUAL'].mean():,.2f}")
    print(f"   Severidad extrapolada mediana: ${siniestrados['SEVERIDAD_EXTRAPOLADA_ANUAL'].median():,.2f}")
    print(f"   Severidad extrapolada máxima: ${siniestrados['SEVERIDAD_EXTRAPOLADA_ANUAL'].max():,.2f}")

print(f"\n6. VIGENCIAS CORTAS")
vigencias_cortas_final = base_final[base_final['DIAS_EXPOSICION'] < 365]
print(f"   Total períodos < 365 días: {len(vigencias_cortas_final):,} ({100*len(vigencias_cortas_final)/len(base_final):.2f}%)")
vigencias_cortas_siniestradas = vigencias_cortas_final[vigencias_cortas_final['TUVO_SINIESTRO'] == 1]
print(f"   Con siniestro: {len(vigencias_cortas_siniestradas):,}")
if len(vigencias_cortas_siniestradas) > 0:
    print(f"   Factor extrapolación promedio: {(365.0 / vigencias_cortas_siniestradas['DIAS_EXPOSICION']).mean():.2f}x")

RESUMEN ESTADÍSTICO FINAL

1. DIMENSIONES
   Registros totales: 738,902
   Pólizas únicas: 212,191
   Combinaciones POLIZA-AMPARO únicas: 400,317

2. DISTRIBUCIÓN TEMPORAL
   Años únicos: 26
   Rango de años: 2000 - 2025

3. EXPOSICIÓN
   Total: $529,110,433,613,753.12
   Promedio: $716,076,602.33
   Mediana: $350,374.91

4. SINIESTRALIDAD
   Registros con siniestro: 2,350
   Registros sin siniestro: 736,552
   Tasa de siniestralidad: 0.32%

5. SEVERIDAD (solo siniestrados)
   Severidad promedio: $4,068,549.35
   Severidad mediana: $0.00
   Severidad máxima: $1,220,498,535.41

   Severidad extrapolada promedio: $4,057,483.13
   Severidad extrapolada mediana: $0.00
   Severidad extrapolada máxima: $1,217,163,839.96

6. VIGENCIAS CORTAS
   Total períodos < 365 días: 66,912 (9.06%)
   Con siniestro: 206
   Factor extrapolación promedio: 3.61x


## Exportar Resultados Finales

In [33]:
print("=" * 80)
print("EXPORTANDO RESULTADOS")
print("=" * 80)

# Exportar base final
base_final.to_csv('../data/tmp/base_final_anual.csv', index=False)
print(f"\n✅ base_final_anual.csv exportado ({len(base_final):,} registros)")

# Exportar siniestros no asignados
if len(siniestros_no_asignados) > 0:
    siniestros_no_asignados_df.to_csv('../data/tmp/siniestros_no_asignados.csv', index=False)
    print(f"✅ siniestros_no_asignados.csv exportado ({len(siniestros_no_asignados):,} registros)")

# Exportar TODOS los siniestros (consolidados y filtrados)
siniestros_consolidado.to_csv('../data/tmp/siniestros_consolidado.csv', index=False)
print(f"✅ siniestros_consolidado.csv exportado ({len(siniestros_consolidado):,} registros)")

print(f"\n" + "=" * 80)
print("TRANSFORMACIÓN COMPLETADA")
print("=" * 80)
print(f"\nArchivos generados:")
print(f"  1. base_final_anual.csv - Base principal para análisis Bühlmann-Straub")
print(f"  2. siniestros_no_asignados.csv - Siniestros sin vigencias correspondientes")
print(f"  3. siniestros_consolidado.csv - Todos los siniestros (período 2010-2019)")

EXPORTANDO RESULTADOS

✅ base_final_anual.csv exportado (738,902 registros)
✅ siniestros_no_asignados.csv exportado (3,162 registros)
✅ siniestros_consolidado.csv exportado (5,746 registros)

TRANSFORMACIÓN COMPLETADA

Archivos generados:
  1. base_final_anual.csv - Base principal para análisis Bühlmann-Straub
  2. siniestros_no_asignados.csv - Siniestros sin vigencias correspondientes
  3. siniestros_consolidado.csv - Todos los siniestros (período 2010-2019)


In [34]:
# =============================================================================
# VALIDACIÓN DE LA NUEVA LÓGICA DE ASIGNACIÓN
# =============================================================================
print("=" * 80)
print("VALIDACIÓN DE LA NUEVA LÓGICA DE ASIGNACIÓN")
print("=" * 80)

# Ejemplo: Tomar una póliza que tuvo siniestro y verificar la lógica
polizas_con_siniestro = base_final[base_final['TUVO_SINIESTRO'] == 1]['POLIZA'].unique()

if len(polizas_con_siniestro) > 0:
    # Seleccionar una póliza de ejemplo que tenga varios períodos
    for poliza_ejemplo in polizas_con_siniestro[:10]:
        periodos_poliza = base_final[base_final['POLIZA'] == poliza_ejemplo].sort_values('VIGENCIA_PERIODO_INICIO')
        if len(periodos_poliza) > 2:  # Buscar una con varios períodos
            break
    
    print(f"\nEjemplo: Póliza {poliza_ejemplo}")
    print(f"Total períodos: {len(periodos_poliza)}")
    
    # Mostrar períodos con sus flags
    print(f"\nDetalle de períodos:")
    for idx, row in periodos_poliza.iterrows():
        siniestro_str = "CON SINIESTRO" if row['TUVO_SINIESTRO'] == 1 else "SIN SINIESTRO"
        tipo_str = "(ORIGINAL)" if row.get('ES_PERIODO_ORIGINAL', False) else "(HEREDADO)" if row['TUVO_SINIESTRO'] == 1 else ""
        severidad_str = f"SEV=${row['SEVERIDAD']:,.0f}" if row['SEVERIDAD'] > 0 else ""
        
        inicio = row['VIGENCIA_PERIODO_INICIO']
        fin = row['VIGENCIA_PERIODO_FIN']
        inicio_str = inicio.strftime('%Y-%m-%d') if hasattr(inicio, 'strftime') else str(inicio)[:10]
        fin_str = fin.strftime('%Y-%m-%d') if hasattr(fin, 'strftime') else str(fin)[:10]
        
        print(f"  P{row['PERIODO']}: {inicio_str} a {fin_str} | {siniestro_str} {tipo_str} {severidad_str}")

print(f"\n" + "=" * 80)
print("RESUMEN DE MEJORA EN ASIGNACIÓN")
print("=" * 80)
print(f"\nTasa de asignación anterior: ~7.85%")
print(f"Tasa de asignación nueva: {((len(siniestros_consolidado) - len(siniestros_no_asignados)) / len(siniestros_consolidado) * 100):.2f}%")
print(f"\nPeríodos con TUVO_SINIESTRO=1: {base_final['TUVO_SINIESTRO'].sum():,}")
print(f"  - Originales (con severidad): {base_final['ES_PERIODO_ORIGINAL'].sum():,}")
print(f"  - Heredados (severidad=0): {(base_final['TUVO_SINIESTRO'] == 1).sum() - base_final['ES_PERIODO_ORIGINAL'].sum():,}")

VALIDACIÓN DE LA NUEVA LÓGICA DE ASIGNACIÓN

Ejemplo: Póliza 1020472
Total períodos: 5

Detalle de períodos:
  P0: 2020-04-01 a 2021-04-01 | CON SINIESTRO (ORIGINAL) SEV=$6,864,750
  P1: 2021-04-02 a 2022-04-02 | CON SINIESTRO (HEREDADO) 
  P2: 2022-04-03 a 2023-04-03 | CON SINIESTRO (HEREDADO) 
  P3: 2023-04-04 a 2024-04-03 | CON SINIESTRO (HEREDADO) 
  P4: 2024-04-04 a 2025-04-04 | CON SINIESTRO (HEREDADO) 

RESUMEN DE MEJORA EN ASIGNACIÓN

Tasa de asignación anterior: ~7.85%
Tasa de asignación nueva: 44.97%

Períodos con TUVO_SINIESTRO=1: 2,350
  - Originales (con severidad): 416
  - Heredados (severidad=0): 1,934


In [ ]:
base_final

,ID_POLIZA_AMPARO_PERIODO,POLIZA,AMPARO,ANIO_POLIZA,PERIODO,VIGENCIA_PERIODO_INICIO,VIGENCIA_PERIODO_FIN,DIAS_EXPOSICION,FRACCION_ANUAL,EXPOSICION_PRORRATEADA,...,COD_SUCURSAL,VIGENCIA_ORIGINAL_INICIO,VIGENCIA_ORIGINAL_FIN,DEPARTAMENTO_SINIESTRO,FECHA_DE_SINIESTRO,FECHA_AVISO,SEVERIDAD,SEVERIDAD_EXTRAPOLADA_ANUAL,TUVO_SINIESTRO,ES_PERIODO_ORIGINAL
2,1020472_PRESTACIONES SOCIALES_2020_0_20200401,1020472,PRESTACIONES SOCIALES,2020,0,2020-04-01,2021-04-01,366,1.002740,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,6864750.0,6.845994e+06,1,True
3,1020472_PRESTACIONES SOCIALES_2021_1_20210402,1020472,PRESTACIONES SOCIALES,2021,1,2021-04-02,2022-04-02,366,1.002740,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
4,1020472_PRESTACIONES SOCIALES_2022_2_20220403,1020472,PRESTACIONES SOCIALES,2022,2,2022-04-03,2023-04-03,366,1.002740,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
5,1020472_PRESTACIONES SOCIALES_2023_3_20230404,1020472,PRESTACIONES SOCIALES,2023,3,2023-04-04,2024-04-03,366,1.002740,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
6,1020472_PRESTACIONES SOCIALES_2024_4_20240404,1020472,PRESTACIONES SOCIALES,2024,4,2024-04-04,2025-04-04,366,1.002740,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411551,100392112_SERIEDAD DE LA OFERTA_2025_0_20250701,100392112,SERIEDAD DE LA OFERTA,2025,0,2025-07-01,2025-11-01,124,0.339726,6.026234e+05,...,10,01/07/2025,01/11/2025,None,None,None,0.0,0.000000e+00,0,False
2411552,100392113_SERIEDAD DE LA OFERTA_2025_0_20250703,100392113,SERIEDAD DE LA OFERTA,2025,0,2025-07-03,2025-10-16,106,0.290411,7.704022e+04,...,10,03/07/2025,16/10/2025,None,None,None,0.0,0.000000e+00,0,False
2411553,100392114_SERIEDAD DE LA OFERTA_2025_0_20250701,100392114,SERIEDAD DE LA OFERTA,2025,0,2025-07-01,2025-09-30,92,0.252055,1.008219e+04,...,10,01/07/2025,30/09/2025,None,None,None,0.0,0.000000e+00,0,False
2411570,100392117_SERIEDAD DE LA OFERTA_2025_0_20250701,100392117,SERIEDAD DE LA OFERTA,2025,0,2025-07-01,2025-10-09,101,0.276712,6.917808e+03,...,10,01/07/2025,09/10/2025,None,None,None,0.0,0.000000e+00,0,False
